In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from skimage.io import imread
from multi_channel_readout import divide_main, extract_signal, read_intensity


CHANNELS = ['cy5', 'TxRed', 'cy3', 'FAM']
BASE_DIR = Path('E:/TMC/PRISM_pipeline/dataset/processed')
RUN_ID = '20230227_test'
src_dir = BASE_DIR / f'{RUN_ID}_processed'
stc_dir = src_dir / 'stitched'
read_dir = src_dir / 'readout'
os.makedirs(read_dir, exist_ok=True)

c:\Users\Mingchuan\anaconda3\envs\cell-typing\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Mingchuan\anaconda3\envs\cell-typing\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Mingchuan\anaconda3\envs\cell-typing\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


ModuleNotFoundError: No module named 'multi_channel_readout'

In [ ]:
image_shape = [1] + list(imread(stc_dir/'cyc_1_cy5.tif').shape)
print('image_shape: ({})'.format(', '.join(map(str, image_shape))))

## spot extraction

In [ ]:
def extract_intensity(pad_x, cut_x, pad_y, cut_y, x_pos, y_pos, x_num, y_num, overlap, channels=CHANNELS):
    os.makedirs(read_dir/'tmp', exist_ok=True)
    image_dict = {}
    coordinate_dict = {}
    for channel in channels:
        coordinate_dict[channel], image_dict[channel] = extract_signal(imread(stc_dir/f'cyc_1_{channel}.tif'), pad_x, cut_x, pad_y, cut_y)
    intensity = pd.concat([read_intensity(image_dict, coordinates) for coordinates in coordinate_dict.values()])
    intensity['X'] = intensity['X'] + pad_x
    intensity['Y'] = intensity['Y'] + pad_y
    intensity.to_csv(read_dir/'tmp'/f'intensity_block_{x_pos}_{y_pos}.csv')

divide_main(func=extract_intensity, shape=image_shape, max_volume=10**8, overlap=500)

image_shape(z, x, y): 1, 33865, 36118
n_tile: 16; 
x_slice_num: 4; y_slice_num: 4;
block_x: 8966; block_y: 9529; overlap: 500;


tile: 100%|██████████| 16/16 [06:06<00:00, 22.93s/it]


## intensity file stitch

In [ ]:
global intensity
intensity = pd.DataFrame()

def stitch_all_block(pad_x, cut_x, pad_y, cut_y, x_pos, y_pos, x_num, y_num, overlap):
    global intensity
    tmp_intensity = pd.read_csv(read_dir/'tmp'/f'intensity_block_{x_pos}_{y_pos}.csv', index_col=0)

    xmin, xmax = pad_x + overlap//2, pad_x + cut_x - overlap//4
    if x_pos == 1: xmin = 0
    elif x_pos == x_num: xmax = pad_x + cut_x
    ymin, ymax = pad_y + overlap//2, pad_y + cut_y - overlap//4
    if y_pos == 1: ymin = 0
    elif y_pos == y_num: ymax = pad_y + cut_y
    tmp_intensity = tmp_intensity[(tmp_intensity['Y']>=ymin)&(tmp_intensity['Y']<=ymax)&
                                  (tmp_intensity['X']>=xmin)&(tmp_intensity['X']<=xmax)]

    intensity = pd.concat([intensity, tmp_intensity])

divide_main(func=stitch_all_block, shape=image_shape, max_volume=10**8, overlap=500)
intensity = intensity.rename(columns={'cy5':'R', 'TxRed':'Ye', 'cy3':'G', 'FAM':'B'})
intensity.to_csv(read_dir / 'intensity_all.csv')

n_tile: 16; 
x_slice_num: 4; y_slice_num: 4;
block_x: 8966; block_y: 9529; overlap: 500;


tile: 100%|██████████| 16/16 [00:02<00:00,  7.14it/s]


## remove duplicate

In [ ]:
from multi_channel_readout import remove_duplicates

intensity['ID'] = intensity['Y'] * 10**7 + intensity['X']
intensity = intensity.drop_duplicates(subset=['Y','X'])

df = intensity[['Y','X','R', 'Ye', 'B', 'G']]
df_reduced = pd.DataFrame()
coordinates = df[['Y','X']].values
coordinates = remove_duplicates(coordinates)
df_reduced = pd.DataFrame(coordinates, columns=['Y','X'])

df_reduced['ID'] = df_reduced['Y'] * 10**7 + df_reduced['X']
intensity = intensity[intensity['ID'].isin(df_reduced['ID'])]

intensity_all = intensity.drop(columns=['ID'])

In [ ]:
# crosstalk elimination
intensity_all['B'] = intensity_all['B'] - intensity_all['G'] * 0.25 
intensity_all = np.maximum(intensity_all, 0)

# Scale
intensity_all['Scaled_R'] = intensity_all['R']
intensity_all['Scaled_Ye'] = intensity_all['Ye']
intensity_all['Scaled_G'] = intensity_all['G'] * 2.5
intensity_all['Scaled_B'] = intensity_all['B'] #* 4/3

intensity_all.to_csv(read_dir/'intensity_all.csv')